# Energy and Vibrational Anharmonicity Landscapes for Halide Double Perovskites

In [ ]:
try:
    from ase.db import connect
except ImportError:
    !pip install ase
    from ase.db import connect

try:
    import plotly 
except ImportError:
    !pip install plotly
    
import plotly.express as px
import os
import sqlite3
import json

from utils import * # this module contains various routines that helps to identify the chemical information in each system

In [ ]:
# Import the database on the disc. First we just retrieve all the keys in the database
# so we can loop through them later to get the relevant records we need for plotting

dbname = os.path.join(os.getcwd(), 'double_halide_pv.db')

all_uids = []
_db = sqlite3.connect(dbname)
cur = _db.cursor()
cur.execute("SELECT * FROM systems")
rows = cur.fetchall()

for row in rows:
    for i in row:
        if 'uid' in str(i):
            this_dict = json.loads(str(i))
            this_uid = this_dict['uid']
            if 'dpv' in this_uid:
                all_uids.append(this_uid)

In [ ]:
#connect to the database using the API from ASE
db = connect(dbname)

In [ ]:
F_data_dict = {'formation_energies': [], 'octahedral_factors': [], 'octahedral_mismatch': [], 'tolerance_factors': [],
                 'A_site_cation': [], 'sigma': [], 'system':[], 'sigma_mode_averaged_300K':[]}
Cl_data_dict = {'formation_energies': [], 'octahedral_factors': [], 'octahedral_mismatch': [], 'tolerance_factors': [],
                 'A_site_cation': [], 'sigma': [], 'system':[], 'sigma_mode_averaged_300K':[]}
Br_data_dict = {'formation_energies': [], 'octahedral_factors': [], 'octahedral_mismatch': [], 'tolerance_factors': [],
                 'A_site_cation': [], 'sigma': [], 'system':[], 'sigma_mode_averaged_300K':[]}
I_data_dict = {'formation_energies': [], 'octahedral_factors': [], 'octahedral_mismatch': [], 'tolerance_factors': [],
                 'A_site_cation': [], 'sigma': [], 'system':[], 'sigma_mode_averaged_300K':[]}

all_data_dict = {'formation_energies': [], 'octahedral_factors': [], 'octahedral_mismatch': [], 'tolerance_factors': [],
                 'A_site_cation': [], 'sigma': [], 'system':[]}

for uid in all_uids:
    try:
        row = db.get(selection=[('uid', '=', uid)])
    except:
        continue
        
    if row is not None:
        atoms = row.toatoms()
        
        try:
            formation_energy = row.key_value_pairs['formation_energy']
            #print('system ' + uid + ' Formation Energy ' + str(formation_energy) + ' eV/atom')
        except KeyError:
            continue

        try:
            sigma = row.key_value_pairs['sigma_300K_single']
            if sigma>=2:
                sigma=None
            #print('system ' + uid + ' sigma ' + str(sigma))
        except KeyError:
            continue
            
        try:
            frequency = row.key_value_pairs['sigma_mode_averaged_300K']
        except KeyError:
            continue
        
        if (formation_energy is not None) and (sigma is not None):
            chemistry, octahedral_factor, octahedral_mismatch, generalised_tolerance_factor = geometric_fingerprint(atoms)
            if octahedral_factor >= octahedral_mismatch + 1 - math.sqrt(2):
                __octahedral_mismatch = octahedral_mismatch
            else:
                __octahedral_mismatch = -1
                
            all_data_dict['system'].append(uid.replace('dpv_',''))    
            all_data_dict['formation_energies'].append(formation_energy)
            all_data_dict['sigma'].append(sigma)
            all_data_dict['octahedral_factors'].append(octahedral_factor)
            all_data_dict['octahedral_mismatch'].append(__octahedral_mismatch)
            all_data_dict['tolerance_factors'].append(generalised_tolerance_factor)
            all_data_dict["A_site_cation"].append(chemistry['A_cation'])
            
            if chemistry['X_anion']=='F':
                F_data_dict['system'].append(uid.replace('dpv_',''))    
                F_data_dict['formation_energies'].append(formation_energy)
                F_data_dict['sigma'].append(sigma)
                F_data_dict['octahedral_factors'].append(octahedral_factor)
                F_data_dict['octahedral_mismatch'].append(__octahedral_mismatch)
                F_data_dict['tolerance_factors'].append(generalised_tolerance_factor)
                F_data_dict["A_site_cation"].append(chemistry['A_cation'])
                F_data_dict['sigma_mode_averaged_300K'].append(frequency)
            elif chemistry['X_anion']=='Cl':
                Cl_data_dict['system'].append(uid.replace('dpv_',''))    
                Cl_data_dict['formation_energies'].append(formation_energy)
                Cl_data_dict['sigma'].append(sigma)
                Cl_data_dict['octahedral_factors'].append(octahedral_factor)
                Cl_data_dict['octahedral_mismatch'].append(__octahedral_mismatch)
                Cl_data_dict['tolerance_factors'].append(generalised_tolerance_factor)
                Cl_data_dict["A_site_cation"].append(chemistry['A_cation'])
                Cl_data_dict['sigma_mode_averaged_300K'].append(frequency)
            elif chemistry['X_anion']=='Br':
                Br_data_dict['system'].append(uid.replace('dpv_',''))    
                Br_data_dict['formation_energies'].append(formation_energy)
                Br_data_dict['sigma'].append(sigma)
                Br_data_dict['octahedral_factors'].append(octahedral_factor)
                Br_data_dict['octahedral_mismatch'].append(__octahedral_mismatch)
                Br_data_dict['tolerance_factors'].append(generalised_tolerance_factor)
                Br_data_dict["A_site_cation"].append(chemistry['A_cation'])
                Br_data_dict['sigma_mode_averaged_300K'].append(frequency)
            elif chemistry['X_anion']=='I':
                I_data_dict['system'].append(uid.replace('dpv_',''))    
                I_data_dict['formation_energies'].append(formation_energy)
                I_data_dict['sigma'].append(sigma)
                I_data_dict['octahedral_factors'].append(octahedral_factor)
                I_data_dict['octahedral_mismatch'].append(__octahedral_mismatch)
                I_data_dict['tolerance_factors'].append(generalised_tolerance_factor)
                I_data_dict["A_site_cation"].append(chemistry['A_cation'])
                I_data_dict['sigma_mode_averaged_300K'].append(frequency)           

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Scatter(x=F_data_dict['octahedral_factors'],
                         y=F_data_dict['tolerance_factors'],
                         name='floride',
                         text=F_data_dict['system'],
                         opacity=0.7))
                         #marker=dict(color=F_data_dict['octahedral_mismatch'], size=8)))
           
fig.add_trace(go.Scatter(x=Cl_data_dict['octahedral_factors'],
                         y=Cl_data_dict['tolerance_factors'],
                         name='chloride',
                         text=Cl_data_dict['system'],
                         opacity=0.7))

fig.add_trace(go.Scatter(x=Br_data_dict['octahedral_factors'],
                         y=Br_data_dict['tolerance_factors'],
                         name='bromide',
                         text=Br_data_dict['system'],
                         opacity=0.7))

fig.add_trace(go.Scatter(x=I_data_dict['octahedral_factors'],
                         y=I_data_dict['tolerance_factors'],
                         name='iodide',
                         text=I_data_dict['system'],
                         opacity=0.7))


fig.update_traces(mode='markers', marker_line_width=2, marker_size=10)


fig.update_xaxes(title_text='Octahedral Factors')
fig.update_yaxes(title_text='Tolerance Factors')


fig.update_layout(
    updatemenus=[go.layout.Updatemenu(
        active=0,
        buttons=list(
            [dict(label = 'All',
                  method = 'update',
                  args = [{'visible': [True, True, True, True]},
                          {'showlegend':True}]),
            dict(label = 'floride',
                  method = 'update',
                  args = [{'visible': [True, False, False, False]},
                          {'showlegend':False}]),
            dict(label = 'chloride',
                  method = 'update',
                  args = [{'visible': [False, True, False, False]},
                          {'showlegend':False}]),
            dict(label = 'bromide',
                  method = 'update',
                  args = [{'visible': [False, False, True, False]},
                          {'showlegend':False}]),
            dict(label = 'iodide',
                  method = 'update',
                  args = [{'visible': [False, False, False, True]},
                          {'showlegend':False}])
            ]))])

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=all_data_dict['octahedral_factors'],
                         y=all_data_dict['tolerance_factors'],
                         name='DHP',
                         text=all_data_dict['system'],
                         opacity=0.7,
                         customdata=all_data_dict['formation_energies'],
                         marker=dict(color=all_data_dict['formation_energies'], size=8,
                                     colorscale='Agsunset_r', colorbar=dict(thickness=30,title='E_f (eV/atom)')),
                         hovertemplate="<br>".join([
                                                    "system: %{text}",
                                                    "Oct. Factor: %{x:.3f}",
                                                    "Tol. Factor: %{y:.3f}",
                                                    "Formation E: %{customdata:.3f} (eV/atom)"])
                        ))


fig.update_traces(
    mode='markers', 
    marker_line_width=0, 
    marker_size=10
)
fig.update_layout(title='Landscape of Halide Double Perovskites with Each Point Colored by Formation Energy')
fig.update_xaxes(title_text='Octahedral Factors')
fig.update_yaxes(title_text='Tolerance Factors')

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=all_data_dict['octahedral_factors'],
                         y=all_data_dict['tolerance_factors'],
                         name='DHP',
                         text=all_data_dict['system'],
                         opacity=0.7,
                         customdata=all_data_dict['octahedral_mismatch'],
                         marker=dict(color=all_data_dict['octahedral_mismatch'], size=8,
                                     colorscale='Aggrnyl', colorbar=dict(thickness=30,title='oct. mismatch')),
                         hovertemplate="<br>".join([
                                                    "system: %{text}",
                                                    "Oct. Factor: %{x:.3f}",
                                                    "Tol. Factor: %{y:.3f}",
                                                    "Oct. Mismatch: %{customdata:.3f}"])
                        ))


fig.update_traces(
    mode='markers', 
    marker_line_width=0, 
    marker_size=10
)
fig.update_layout(title='Landscape of Halide Double Perovskites with Each Point Colored by Octahderal Mismatch')
fig.update_xaxes(title_text='Octahedral Factors')
fig.update_yaxes(title_text='Tolerance Factors')

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=all_data_dict['octahedral_factors'],
                         y=all_data_dict['tolerance_factors'],
                         name='DHP',
                         text=all_data_dict['system'],
                         opacity=0.7,
                         customdata=all_data_dict['sigma'],
                         marker=dict(color=all_data_dict['sigma'], size=8,
                                     colorscale='RdBu_r', colorbar=dict(thickness=30,title='sigma')),
                         hovertemplate="<br>".join([
                                                    "system: %{text}",
                                                    "Oct. Factor: %{x:.3f}",
                                                    "Tol. Factor: %{y:.3f}",
                                                    "Sigma: %{customdata:.3f}"])
                        ))


fig.update_traces(
    mode='markers', 
    marker_line_width=0, 
    marker_size=10
)
fig.update_layout(title='Landscape of Halide Double Perovskites with Each Point Colored by Anharmonic Score (Sigma)')
fig.update_xaxes(title_text='Octahedral Factors')
fig.update_yaxes(title_text='Tolerance Factors')

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Scatter(x=F_data_dict['formation_energies'],
                         y=F_data_dict['sigma'],
                         name='floride',
                         text=F_data_dict['system'],
                         opacity=0.7))
                         #marker=dict(color=F_data_dict['octahedral_mismatch'], size=8)))
           
fig.add_trace(go.Scatter(x=Cl_data_dict['formation_energies'],
                         y=Cl_data_dict['sigma'],
                         name='chloride',
                         text=Cl_data_dict['system'],
                         opacity=0.7))

fig.add_trace(go.Scatter(x=Br_data_dict['formation_energies'],
                         y=Br_data_dict['sigma'],
                         name='bromide',
                         text=Br_data_dict['system'],
                         opacity=0.7))

fig.add_trace(go.Scatter(x=I_data_dict['formation_energies'],
                         y=I_data_dict['sigma'],
                         name='iodide',
                         text=I_data_dict['system'],
                         opacity=0.7))


fig.update_traces(mode='markers', marker_line_width=2, marker_size=10)


fig.update_xaxes(title_text='Formation Energies (eV/atom)')
fig.update_yaxes(title_text='Anharmonic Scores')


fig.update_layout(
    updatemenus=[go.layout.Updatemenu(
        active=0,
        buttons=list(
            [dict(label = 'All',
                  method = 'update',
                  args = [{'visible': [True, True, True, True]},
                          {'showlegend':True}]),
            dict(label = 'floride',
                  method = 'update',
                  args = [{'visible': [True, False, False, False]},
                          {'showlegend':False}]),
            dict(label = 'chloride',
                  method = 'update',
                  args = [{'visible': [False, True, False, False]},
                          {'showlegend':False}]),
            dict(label = 'bromide',
                  method = 'update',
                  args = [{'visible': [False, False, True, False]},
                          {'showlegend':False}]),
            dict(label = 'iodide',
                  method = 'update',
                  args = [{'visible': [False, False, False, True]},
                          {'showlegend':False}])
            ]))])

fig.update_layout(title='Vibrational Anharmonicity against Formation Energies for Halide Double Perovskites')
